From the main directory "Ontolearn", run the commands for NCES data mentioned [here](https://ontolearn-docs-dice-group.netlify.app/usage/02_installation#download-external-files) to download pretrained models and datasets.

In [1]:
from ontolearn.concept_learner import NCES
from ontolearn.knowledge_base import KnowledgeBase
from owlapy.parser import DLSyntaxParser
from owlapy.render import DLSyntaxObjectRenderer
import sys
sys.path.append("examples/")
from ontolearn.metrics import F1
from quality_functions import quality
import time
import random

/home/nkouagou/.conda/envs/onto/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
nces = NCES(knowledge_base_path="../NCESData/family/family.owl", quality_func=F1(), num_predictions=100, learner_name="SetTransformer",
     path_of_embeddings="../NCESData/family/embeddings/ConEx_entity_embeddings.csv", load_pretrained=True, max_length=48, proj_dim=128, rnn_n_layers=2, drop_prob=0.1, num_heads=4, num_seeds=1, num_inds=32, pretrained_model_name="SetTransformer")

/home/nkouagou/.conda/envs/onto/lib/python3.10/site-packages/torch/cuda/__init__.py:141: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 6050). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


In [3]:
KB = KnowledgeBase(path=nces.knowledge_base_path)

In [4]:
dl_syntax_renderer = DLSyntaxObjectRenderer()

In [5]:
atomic_classes = [dl_syntax_renderer.render(a) for a in KB.ontology.classes_in_signature()]

In [6]:
atomic_classes

['Brother',
 'Male',
 'PersonWithASibling',
 'Child',
 'Person',
 'Daughter',
 'Female',
 'Father',
 'Parent',
 'Grandchild',
 'Granddaughter',
 'Grandfather',
 'Grandparent',
 'Grandmother',
 'Grandson',
 'Mother',
 'Sister',
 'Son']

In [7]:
dl_parser = DLSyntaxParser(nces.kb_namespace)

In [8]:
brother = dl_parser.parse('Brother')
daughter = dl_parser.parse('Daughter')

#### Input examples can be sets or lists

In [9]:
pos = set(KB.individuals(brother)).union(set(KB.individuals(daughter)))

In [10]:
neg = set(KB.individuals())-set(pos)

In [11]:
len(neg)

120

#### Prediction with SetTransformer (default model)

In [12]:
t0 = time.time()
node = list(nces.fit(pos, neg).best_predictions)[0]
t1 = time.time()
print("\nDuration: ", t1-t0, " seconds")


Duration:  0.5292303562164307  seconds


In [13]:
len(set(KB.individuals(node.concept)))

82

In [14]:
node

<class 'ontolearn.search.NCESNode'> at 0x9fe2fe8	Brother ⊔ Sister ⊔ Daughter	Quality:1.0	Length:5	|Indv.|:82

In [15]:
quality(KB, node.concept, pos, neg)

Accuracy: 100.0%
Precision: 100.0%
Recall: 100.0%
F1: 100.0%


(100.0, 100.0, 100.0, 100.0)

### Ensemble prediction

In [16]:
nces.pretrained_model_name = ['SetTransformer','GRU','LSTM']
nces.refresh()
t0 = time.time()
node = list(nces.fit(pos, neg).best_predictions)[0]
t1 = time.time()
print("\nDuration: ", t1-t0, " seconds")


Duration:  0.6503381729125977  seconds


In [17]:
quality(KB, node.concept, pos, neg)

Accuracy: 100.0%
Precision: 100.0%
Recall: 100.0%
F1: 100.0%


(100.0, 100.0, 100.0, 100.0)

In [18]:
node

<class 'ontolearn.search.NCESNode'> at 0x9fe2c38	Brother ⊔ Daughter	Quality:1.0	Length:3	|Indv.|:82

In [19]:
nces.best_predictions[:5]

[<class 'ontolearn.search.NCESNode'> at 0x9fe2c38	Brother ⊔ Daughter	Quality:1.0	Length:3	|Indv.|:82,
 <class 'ontolearn.search.NCESNode'> at 0xa05c220	Brother ⊔ Daughter	Quality:1.0	Length:3	|Indv.|:82,
 <class 'ontolearn.search.NCESNode'> at 0x9fe34b0	Brother ⊔ Daughter	Quality:1.0	Length:3	|Indv.|:82,
 <class 'ontolearn.search.NCESNode'> at 0x9fe3ca8	PersonWithASibling ⊔ Daughter ⊔ Sister	Quality:1.0	Length:5	|Indv.|:82,
 <class 'ontolearn.search.NCESNode'> at 0x9fe3e08	Brother ⊔ Daughter	Quality:1.0	Length:3	|Indv.|:82]

### Complex learning problems, potentially without an exact solution

#### First learning problem

In [20]:
all_individuals = set(KB.individuals())
pos = set(random.sample(list(all_individuals), 150))
remaining = all_individuals-pos
neg = set(random.sample(list(remaining), min(100, len(remaining))))

In [21]:
nces.pretrained_model_name

['SetTransformer', 'GRU', 'LSTM']

In [22]:
t0 = time.time()
node = list(nces.fit(pos, neg).best_predictions)[0]
t1 = time.time()
print("\nDuration: ", t1-t0, " seconds")


Duration:  0.6927070617675781  seconds


In [23]:
node

<class 'ontolearn.search.NCESNode'> at 0xcb9e860	Person ⊔ Son	Quality:0.85227	Length:3	|Indv.|:202

In [24]:
quality(KB, node.concept, pos, neg)

Accuracy: 74.25699999999999%
Precision: 74.25699999999999%
Recall: 100.0%
F1: 85.227%


(74.25699999999999, 74.25699999999999, 100.0, 85.227)

#### Second learning problem

In [25]:
pos = set(random.sample(list(all_individuals), 80))
remaining = all_individuals-pos
neg = set(random.sample(list(remaining), min(150, len(remaining))))

In [26]:
t0 = time.time()
node = list(nces.fit(pos, neg).best_predictions)[0]
t1 = time.time()
print("\nDuration: ", t1-t0, " seconds")


Duration:  0.7965126037597656  seconds


In [27]:
quality(KB, node.concept, pos, neg)

Accuracy: 57.921%
Precision: 47.934%
Recall: 72.5%
F1: 57.711%


(57.921, 47.934, 72.5, 57.711)

## Important note

- Each of the synthesized expressions are not present in the knowledge base.
- NCES synthesizes solutions by leveraging its experience on the training data.
- The inputs (positive/negative examples) need not be balanced
- NCES can solve multiple learning problems at the same time (through broadcasting on matrix operations in its neural network component), see nces_notebook1.ipynb
- Since LSTM and GRU are not permutation-equivariant, we can get different but closely related solutions by shuflling the input examples for these architectures. For this, one needs to instantiate the NCES class with the attribute "sorted_examples=False" which is the case by default.